In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf

from Hypers import *
from Utils import *
from Preprocessing import *
from Generator import *
from Models import *

In [ ]:
MAX_CLICK = 50
MAX_TITLE = 30
filter_num = 10

In [ ]:
def getNewsCnt(train_session,news_index):
    max_cnt=0
    news_freq_cnt=np.zeros(len(news_index))
    for sess in train_session:
        clicks = sess[0]
        for c in clicks:              
            news_freq_cnt[news_index[c]] += 1
            max_cnt = max(max_cnt,news_freq_cnt[news_index[c]])
    return news_freq_cnt,max_cnt

In [3]:
data_path = ' '
word_embedding_path = ' '

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'train_behaviors.tsv')
valid_news_file = os.path.join(data_path, 'train', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'train', r'val_behaviors.tsv')
test_news_file = os.path.join(data_path, 'valid', r'news.tsv')
test_behaviors_file = os.path.join(data_path, 'valid', r'test_behaviors.tsv')

In [6]:
news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict = read_news([train_news_file,test_news_file])
news_title,news_vert,news_subvert,news_entity,news_content=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict,content_dict,entity_dict)
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)
news_info = np.concatenate([news_title],axis=-1)

In [ ]:
train_session = read_train_clickhistory(news_index,data_path,'train/train_behaviors.tsv',filter_num)
train_user = parse_user(news_index,train_session)
news_freq_cnt,max_cnt = getNewsCnt(train_session,news_index)
train_sess, train_user_id, train_label = get_train_input(news_index,train_session)
train_generator = get_hir_train_generator(news_info,train_user['click'],train_user_id,train_sess,train_label,64,news_freq_cnt)

test_session = read_test_clickhistory(news_index,data_path,'valid/test_behaviors.tsv',filter_num)
test_user = parse_user(news_index,test_session)
test_impressions, test_userids= get_test_input(news_index,test_session)


model,news_encoder,user_encoder, = MCCM(title_word_embedding_matrix)
model.fit_generator(train_generator,epochs=8,verbose=1)

news_scoring = news_encoder.predict(news_info,verbose=1)
test_generator = get_user_generator(news_info,test_userids,test_user['click'],32)
test_user_scoring = user_encoder.predict_generator(test_generator,verbose=1)
AUC, MRR, nDCG5, nDCG10 = evaluate(test_impressions,news_scoring,test_user_scoring)